
# Анализ бизнес-показателей для развлекательного проекта



---
## Контекст


Мы делаем проект для развлекательного приложения Procrastinate Pro+. Несколько прошлых месяцев бизнес постоянно нес убытки - в привлечение пользователей была вложена куча денег, а толку никакого. Нам нужно разобраться в причинах этой ситуации.


## Задача

У нас в распоряжении есть лог сервера с данными о посещениях приложения новыми пользователями, зарегистрировавшимися в период с 2019-05-01 по 2019-10-27, выгрузка их покупок за этот период, а также статистика рекламных расходов. Нам предстоит изучить, как люди пользуются продуктом, когда они начинают покупать, сколько денег приносит каждый клиент, когда он окупается и какие факторы отрицательно влияют на привлечение пользователей.


---
## План работы


* `Шаг 1.` Загрузим данные и подготовим их к анализу

Загрузим данные о визитах, заказах и расходах в переменные. Оптимизируем данные для анализа. Убедитесь, что тип данных в каждой колонке — правильный. 
 
* `Шаг 2.` Зададим функции для расчета и анализа LTV, ROI, удержания и конверсии


* `Шаг 3.` Проведем исследовательский анализ данных

Построим профили пользователей. Определим минимальную и максимальную дату привлечения пользователей.

Выясним:

>  - Из каких стран приходят посетители? Какие страны дают больше всего платящих пользователей?
>  - Какими устройствами они пользуются? С каких устройств чаще всего заходят платящие пользователи?
>  - По каким рекламным каналам шло привлечение пользователей? Какие каналы приносят больше всего платящих пользователей?.


* `Шаг 4.` Маркетинг

 Выясним:

> - Сколько денег потратили? Всего / на каждый источник / по времени
> - Сколько в среднем стоило привлечение одного покупателя из каждого источника?

* `Шаг 5.` Оценим окупаемость рекламы для привлечения пользователей

С помощью LTV и ROI:

> - Проанализируем общую окупаемость рекламы;
> - Проанализируем окупаемость рекламы с разбивкой по устройствам;
> - Проанализируем окупаемость рекламы с разбивкой по странам;
> - Проанализируем окупаемость рекламы с разбивкой по рекламным каналам.

Опишем проблемы, которые мы обнаружили. Ответим на вопросы:

> - Окупается ли реклама, направленная на привлечение пользователей в целом? 
> - Какие устройства, страны и рекламные каналы могут оказывать негативное влияние на окупаемость рекламы?
> - Чем могут быть вызваны проблемы окупаемости? Изучим конверсию и удержание с разбивкой по устройствам, странам, рекламным каналам.

Опишем возможные причины обнаруженных проблем и сформируем рекомендации для рекламного отдела. При решении этого шага считаем, что мы смотрим данные 1-го ноября 2019 года и что в нашей организации принято считать, что окупаемость должна наступать не позднее, чем через 2 недели после привлечения пользователей.

* `Шаг 6.` Выводы

> - Выделим причины неэффективности привлечения пользователей;
> - Сформируем рекомендации для отдела маркетинга для повышения эффективности.


---
##  Описание данных

Таблица `visits_log_short` (лог сервера с информацией о посещениях сайта):

   * `User Id` — уникальный идентификатор пользователя
   * `Device` — категория устройства пользователя
   * `Session start` — дата и время начала сессии
   * `Session End` — дата и время окончания сессии
   * `Channel` — идентификатор рекламного источника, из которого пришел пользователь
   * `Region` - страна пользователя

Таблица `orders_log_short` (информация о заказах):

   * `User Id` — уникальный id пользователя, который сделал заказ
   * `Event Dt` — дата и время покупки
   * `Revenue` — выручка

Таблица `costs_short` (информация о затратах на маркетинг):

   * `Channel` — идентификатор рекламного источника
   * `Dt` — дата
   * `Costs` — затраты на этот рекламный источник в этот день

Импортируем библиотеки.

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import decimal
import numpy as np
from matplotlib import pyplot as plt

Сохраняем данные в переменные.

In [3]:
try:
    visits = pd.read_csv('/datasets/visits_info_short.csv')
    orders = pd.read_csv('/datasets/orders_info_short.csv')
    costs = pd.read_csv('/datasets/costs_info_short.csv')
except:
    visits = pd.read_csv('/Users/ivan_miroshnichenko/Downloads/datasets/visits_info_short.csv')
    orders = pd.read_csv('/Users/ivan_miroshnichenko/Downloads/datasets/orders_info_short.csv')
    costs = pd.read_csv('/Users/ivan_miroshnichenko/Downloads/datasets/costs_info_short.csv')
    

Смотрим в первый раз на данные.

In [ ]:
display(visits.sample(20), orders.sample(20), costs.sample(20))

Приведем названия столбцов к снэйк кейсу и нижнему регистру.

In [ ]:
visits = visits.rename(columns={'Session End': 'session_end', 'Session Start': 'session_start', 'User Id': 'user_id'})
visits.columns = visits.columns.str.lower()

orders = orders.rename(columns={'User Id': 'user_id','Event Dt': 'event_dt'})
orders.columns = orders.columns.str.lower()

costs.columns = costs.columns.str.lower()

Взглянем на изменения.

In [ ]:
display(visits.sample(20))
display(visits.info())
display(orders.sample(20))
display(orders.info())
display(costs.sample(20))
display(costs.info())

Создадим столбцы с датой начала сессии, неделей когда была начата сессия и месяцем сессии. Приведем все столбцы, где храним даты к формату datetime.

In [ ]:
visits['session_start'] = pd.to_datetime(visits['session_start'])
visits['session_end'] = pd.to_datetime(visits['session_end'])
visits['session_date'] = visits['session_start'].dt.date
visits['session_week'] = visits['session_start'].dt.isocalendar().week
visits['session_month'] = visits['session_start'].dt.month
visits['session_duration'] = visits['session_end'] - visits['session_start']
visits['session_duration'] = visits['session_duration'].apply(lambda x: decimal.Decimal(x.seconds) // 60)
visits['session_duration'] = visits['session_duration'].astype('int')
visits['session_date'] = pd.to_datetime(visits['session_date'])

costs['dt'] = pd.to_datetime(costs['dt']).dt.date

orders['event_dt'] = pd.to_datetime(orders['event_dt'])

Взглянем на обновленные данные.

In [ ]:
display(
        visits.sample(20),
        visits.info(),
        orders.sample(20),
        orders.info(),
        costs.sample(20),
        costs.info()
        )

Смотрим на наличие пропусков и дубликатов.

In [ ]:
print(
        'Количество пропусков visits -', costs.isnull().sum(),
        'Количество пропусков orders -', orders.isnull().sum(),
        'Количество пропусков costs -', visits.isnull().sum()
        )

print(
        'Количество дубликатов visits -', visits.duplicated().sum(),
        'Количество дубликатов orders -', orders.duplicated().sum(),
        'Количество дубликатов costs -', costs.duplicated().sum()
        )

Качество исходных данных отличное, так как отсутствуют дубликаты и пропуски.

# Функции для расчета и анализа LTV, ROI, удержания и конверсии

Зададим функции необходимые нам для анализа.

Функция для формирования пользовательских профилей.

In [ ]:
def get_profiles(sessions, orders, ad_costs):

    # находим параметры первых посещений
    profiles = (
        sessions.sort_values(by=['user_id', 'session_start'])
        .groupby('user_id')
        .agg(
            {
                'session_start': 'first',
                'channel': 'first',
                'device': 'first',
                'region': 'first',
            }
        )
        .rename(columns={'session_start': 'first_ts'})
        .reset_index()
    )

    # для когортного анализа определяем дату первого посещения
    # и первый день месяца, в который это посещение произошло
    profiles['dt'] = profiles['first_ts'].dt.date
    profiles['month'] = profiles['first_ts'].astype('datetime64[M]')

    # добавляем признак платящих пользователей
    profiles['payer'] = profiles['user_id'].isin(orders['user_id'].unique())

    # считаем количество уникальных пользователей
    # с одинаковыми источником и датой привлечения
    new_users = (
        profiles.groupby(['dt', 'channel'])
        .agg({'user_id': 'nunique'})
        .rename(columns={'user_id': 'unique_users'})
        .reset_index()
    )

    # объединяем траты на рекламу и число привлечённых пользователей
    ad_costs = ad_costs.merge(new_users, on=['dt', 'channel'], how='left')

    # делим рекламные расходы на число привлечённых пользователей
    ad_costs['acquisition_cost'] = ad_costs['costs'] / ad_costs['unique_users']

    # добавляем стоимость привлечения в профили
    profiles = profiles.merge(
        ad_costs[['dt', 'channel', 'acquisition_cost']],
        on=['dt', 'channel'],
        how='left',
    )

    # стоимость привлечения органических пользователей равна нулю
    profiles['acquisition_cost'] = profiles['acquisition_cost'].fillna(0)

    return profiles
 

Функция для расчёта удержания.

In [ ]:
def get_retention(
    profiles,
    sessions,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # добавляем столбец payer в передаваемый dimensions список
    dimensions = ['payer'] + dimensions

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')

    # собираем «сырые» данные для расчёта удержания
    result_raw = result_raw.merge(
        sessions[['user_id', 'session_start']], on='user_id', how='left'
    )
    result_raw['lifetime'] = (
        result_raw['session_start'] - result_raw['first_ts']
    ).dt.days

    # функция для группировки таблицы по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):
        result = df.pivot_table(
            index=dims, columns='lifetime', values='user_id', aggfunc='nunique'
        )
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        result = result.div(result['cohort_size'], axis=0)
        result = result[['cohort_size'] + list(range(horizon_days))]
        result['cohort_size'] = cohort_sizes
        return result

    # получаем таблицу удержания
    result_grouped = group_by_dimensions(result_raw, dimensions, horizon_days)

    # получаем таблицу динамики удержания
    result_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    # возвращаем обе таблицы и сырые данные
    return result_raw, result_grouped, result_in_time 

Функция для расчета конверсии.

In [ ]:
def get_conversion(
    profiles,
    purchases,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')

    # определяем дату и время первой покупки для каждого пользователя
    first_purchases = (
        purchases.sort_values(by=['user_id', 'event_dt'])
        .groupby('user_id')
        .agg({'event_dt': 'first'})
        .reset_index()
    )

    # добавляем данные о покупках в профили
    result_raw = result_raw.merge(
        first_purchases[['user_id', 'event_dt']], on='user_id', how='left'
    )

    # рассчитываем лайфтайм для каждой покупки
    result_raw['lifetime'] = (
        result_raw['event_dt'] - result_raw['first_ts']
    ).dt.days

    # группируем по cohort, если в dimensions ничего нет
    if len(dimensions) == 0:
        result_raw['cohort'] = 'All users' 
        dimensions = dimensions + ['cohort']

    # функция для группировки таблицы по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):
        result = df.pivot_table(
            index=dims, columns='lifetime', values='user_id', aggfunc='nunique'
        )
        result = result.fillna(0).cumsum(axis = 1)
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        # делим каждую «ячейку» в строке на размер когорты
        # и получаем conversion rate
        result = result.div(result['cohort_size'], axis=0)
        result = result[['cohort_size'] + list(range(horizon_days))]
        result['cohort_size'] = cohort_sizes
        return result

    # получаем таблицу конверсии
    result_grouped = group_by_dimensions(result_raw, dimensions, horizon_days)

    # для таблицы динамики конверсии убираем 'cohort' из dimensions
    if 'cohort' in dimensions: 
        dimensions = []

    # получаем таблицу динамики конверсии
    result_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    # возвращаем обе таблицы и сырые данные
    return result_raw, result_grouped, result_in_time 

Функция для расчёта LTV и ROI.

In [ ]:
def get_ltv(
    profiles,
    purchases,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')
    # добавляем данные о покупках в профили
    result_raw = result_raw.merge(
        purchases[['user_id', 'event_dt', 'revenue']], on='user_id', how='left'
    )
    # рассчитываем лайфтайм пользователя для каждой покупки
    result_raw['lifetime'] = (
        result_raw['event_dt'] - result_raw['first_ts']
    ).dt.days
    # группируем по cohort, если в dimensions ничего нет
    if len(dimensions) == 0:
        result_raw['cohort'] = 'All users'
        dimensions = dimensions + ['cohort']

    # функция группировки по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):
        # строим «треугольную» таблицу выручки
        result = df.pivot_table(
            index=dims, columns='lifetime', values='revenue', aggfunc='sum'
        )
        # находим сумму выручки с накоплением
        result = result.fillna(0).cumsum(axis=1)
        # вычисляем размеры когорт
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        # объединяем размеры когорт и таблицу выручки
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        # считаем LTV: делим каждую «ячейку» в строке на размер когорты
        result = result.div(result['cohort_size'], axis=0)
        # исключаем все лайфтаймы, превышающие горизонт анализа
        result = result[['cohort_size'] + list(range(horizon_days))]
        # восстанавливаем размеры когорт
        result['cohort_size'] = cohort_sizes

        # собираем датафрейм с данными пользователей и значениями CAC, 
        # добавляя параметры из dimensions
        cac = df[['user_id', 'acquisition_cost'] + dims].drop_duplicates()

        # считаем средний CAC по параметрам из dimensions
        cac = (
            cac.groupby(dims)
            .agg({'acquisition_cost': 'mean'})
            .rename(columns={'acquisition_cost': 'cac'})
        )

        # считаем ROI: делим LTV на CAC
        roi = result.div(cac['cac'], axis=0)

        # удаляем строки с бесконечным ROI
        roi = roi[~roi['cohort_size'].isin([np.inf])]

        # восстанавливаем размеры когорт в таблице ROI
        roi['cohort_size'] = cohort_sizes

        # добавляем CAC в таблицу ROI
        roi['cac'] = cac['cac']

        # в финальной таблице оставляем размеры когорт, CAC
        # и ROI в лайфтаймы, не превышающие горизонт анализа
        roi = roi[['cohort_size', 'cac'] + list(range(horizon_days))]

        # возвращаем таблицы LTV и ROI
        return result, roi

    # получаем таблицы LTV и ROI
    result_grouped, roi_grouped = group_by_dimensions(
        result_raw, dimensions, horizon_days
    )

    # для таблиц динамики убираем 'cohort' из dimensions
    if 'cohort' in dimensions:
        dimensions = []

    # получаем таблицы динамики LTV и ROI
    result_in_time, roi_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    return (
        result_raw,  # сырые данные
        result_grouped,  # таблица LTV
        result_in_time,  # таблица динамики LTV
        roi_grouped,  # таблица ROI
        roi_in_time,  # таблица динамики ROI
    ) 

Функция для визуализации удержания.


In [ ]:
def plot_retention(retention, retention_history, horizon, window=7):

    # задаём размер сетки для графиков
    plt.figure(figsize=(15, 10))

    # исключаем размеры когорт и удержание первого дня
    retention = retention.drop(columns=['cohort_size', 0])
    # в таблице динамики оставляем только нужный лайфтайм
    retention_history = retention_history.drop(columns=['cohort_size'])[
        [horizon - 1]
    ]

    # если в индексах таблицы удержания только payer,
    # добавляем второй признак — cohort
    if retention.index.nlevels == 1:
        retention['cohort'] = 'All users'
        retention = retention.reset_index().set_index(['cohort', 'payer'])

    # в таблице графиков — два столбца и две строки, четыре ячейки
    # в первой строим кривые удержания платящих пользователей
    ax1 = plt.subplot(2, 2, 1)
    retention.query('payer == True').droplevel('payer').T.plot(
        grid=True, ax=ax1
    )
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('Удержание платящих пользователей')

    # во второй ячейке строим кривые удержания неплатящих
    # вертикальная ось — от графика из первой ячейки
    ax2 = plt.subplot(2, 2, 2, sharey=ax1)
    retention.query('payer == False').droplevel('payer').T.plot(
        grid=True, ax=ax2
    )
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('Удержание неплатящих пользователей')

    # в третьей ячейке — динамика удержания платящих
    ax3 = plt.subplot(2, 2, 3)
    # получаем названия столбцов для сводной таблицы
    columns = [
        name
        for name in retention_history.index.names
        if name not in ['dt', 'payer']
    ]
    # фильтруем данные и строим график
    filtered_data = retention_history.query('payer == True').pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax3)
    plt.xlabel('Дата привлечения')
    plt.title(
        'Динамика удержания платящих пользователей на {}-й день'.format(
            horizon
        )
    )

    # в чётвертой ячейке — динамика удержания неплатящих
    ax4 = plt.subplot(2, 2, 4, sharey=ax3)
    # фильтруем данные и строим график
    filtered_data = retention_history.query('payer == False').pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax4)
    plt.xlabel('Дата привлечения')
    plt.title(
        'Динамика удержания неплатящих пользователей на {}-й день'.format(
            horizon
        )
    )
    
    plt.tight_layout()
    plt.show() 

Функция для визуализации LTV и ROI.

In [ ]:
def plot_ltv_roi(ltv, ltv_history, roi, roi_history, horizon, window=7):

    # задаём сетку отрисовки графиков
    plt.figure(figsize=(20, 10))

    # из таблицы ltv исключаем размеры когорт
    ltv = ltv.drop(columns=['cohort_size'])
    # в таблице динамики ltv оставляем только нужный лайфтайм
    ltv_history = ltv_history.drop(columns=['cohort_size'])[[horizon - 1]]

    # стоимость привлечения запишем в отдельный фрейм
    cac_history = roi_history[['cac']]

    # из таблицы roi исключаем размеры когорт и cac
    roi = roi.drop(columns=['cohort_size', 'cac'])
    # в таблице динамики roi оставляем только нужный лайфтайм
    roi_history = roi_history.drop(columns=['cohort_size', 'cac'])[
        [horizon - 1]
    ]

    # первый график — кривые ltv
    ax1 = plt.subplot(2, 3, 1)
    ltv.T.plot(grid=True, ax=ax1)
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('LTV')

    # второй график — динамика ltv
    ax2 = plt.subplot(2, 3, 2, sharey=ax1)
    # столбцами сводной таблицы станут все столбцы индекса, кроме даты
    columns = [name for name in ltv_history.index.names if name not in ['dt']]
    filtered_data = ltv_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax2)
    plt.xlabel('Дата привлечения')
    plt.title('Динамика LTV пользователей на {}-й день'.format(horizon))

    # третий график — динамика cac
    ax3 = plt.subplot(2, 3, 3, sharey=ax1)
    # столбцами сводной таблицы станут все столбцы индекса, кроме даты
    columns = [name for name in cac_history.index.names if name not in ['dt']]
    filtered_data = cac_history.pivot_table(
        index='dt', columns=columns, values='cac', aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax3)
    plt.xlabel('Дата привлечения')
    plt.title('Динамика стоимости привлечения пользователей')

    # четвёртый график — кривые roi
    ax4 = plt.subplot(2, 3, 4)
    roi.T.plot(grid=True, ax=ax4)
    plt.axhline(y=1, color='red', linestyle='--', label='Уровень окупаемости')
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('ROI')

    # пятый график — динамика roi
    ax5 = plt.subplot(2, 3, 5, sharey=ax4)
    # столбцами сводной таблицы станут все столбцы индекса, кроме даты
    columns = [name for name in roi_history.index.names if name not in ['dt']]
    filtered_data = roi_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax5)
    plt.axhline(y=1, color='red', linestyle='--', label='Уровень окупаемости')
    plt.xlabel('Дата привлечения')
    plt.title('Динамика ROI пользователей на {}-й день'.format(horizon))

    plt.tight_layout()
    plt.show() 

In [ ]:
def filter_data(df, window):
    # для каждого столбца применяем скользящее среднее
    for column in df.columns.values:
        df[column] = df[column].rolling(window).mean() 
    return df 

Функция для визуализации конверсии.

In [ ]:
def plot_conversion(conversion, conversion_history, horizon, window=7):

    # задаём размер сетки для графиков
    plt.figure(figsize=(15, 5))

    # исключаем размеры когорт
    conversion = conversion.drop(columns=['cohort_size'])
    # в таблице динамики оставляем только нужный лайфтайм
    conversion_history = conversion_history.drop(columns=['cohort_size'])[
        [horizon - 1]
    ]

    # первый график — кривые конверсии
    ax1 = plt.subplot(1, 2, 1)
    conversion.T.plot(grid=True, ax=ax1)
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('Конверсия пользователей')

    # второй график — динамика конверсии
    ax2 = plt.subplot(1, 2, 2, sharey=ax1)
    columns = [
        # столбцами сводной таблицы станут все столбцы индекса, кроме даты
        name for name in conversion_history.index.names if name not in ['dt']
    ]
    filtered_data = conversion_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax2)
    plt.xlabel('Дата привлечения')
    plt.title('Динамика конверсии пользователей на {}-й день'.format(horizon))

    plt.tight_layout()
    plt.show() 

# Шаг 3. Исследовательский анализ данных


Построим профили пользователей.


In [ ]:
profiles = get_profiles(visits, orders, costs)
profiles.sample(20)

Определяем минимальную и максимальную даты привлечения пользователей.

In [ ]:
print('Минимальную дата привлечения пользователей -', profiles['first_ts'].min())
print('Максимальная дата привлечения пользователей -', profiles['first_ts'].max())

Минимальная и максимальная дата привлечения пользователя соответствуют условиям ТЗ. Можно проводить анализ далее.

Определим количество платящих пользователей для каждого региона, и какой процент они составляют от общего числа пользователей в регионе.

In [ ]:
users_per_region = profiles.pivot_table(index='region', values='payer', aggfunc='count')
payer_per_region = profiles.query('payer == True').pivot_table(index='region', values='payer', aggfunc='count')
payer_per_region = users_per_region.merge(payer_per_region,how='inner', on='region')
payer_per_region['%'] = payer_per_region['payer_y'] / payer_per_region['payer_x']
payer_per_region = payer_per_region.rename(columns={'payer_x':'total_user', 'payer_y':'payer', '%':'percent_of_total'})
payer_per_region.style.format({('percent_of_total'):'{:.2%}'})

Определим количество платящих пользователей для каждого канала привлечения, и какой процент они составляют от общего числа пользователей из этого канала.

In [ ]:
users_per_channel = profiles.pivot_table(index='channel', values='payer', aggfunc='count')
payer_per_channel = profiles.query('payer == True').pivot_table(index='channel', values='payer', aggfunc='count')
payer_per_channel = users_per_channel.merge(payer_per_channel,how='inner', on='channel')
payer_per_channel['%'] = payer_per_channel['payer_y'] / payer_per_channel['payer_x']
payer_per_channel = payer_per_channel.rename(columns={'payer_x':'total_user', 'payer_y':'payer', '%':'percent_of_total'})
payer_per_channel.style.format({('percent_of_total'):'{:.2%}'})

Определим количество платящих пользователей для каждого типа устройств, и какой процент они составляют от общего числа пользователей каждого типа устройств.

In [ ]:
users_per_device = profiles.pivot_table(index='device', values='payer', aggfunc='count')
payer_per_device = profiles.query('payer == True').pivot_table(index='device', values='payer', aggfunc='count')
payer_per_device = users_per_device.merge(payer_per_device,how='inner', on='device')
payer_per_device['%'] = payer_per_device['payer_y'] / payer_per_device['payer_x']
payer_per_device = payer_per_device.rename(columns={'payer_x':'total_user', 'payer_y':'payer', '%':'percent_of_total'})
payer_per_device.style.format({('percent_of_total'):'{:.2%}'})

Получается, что больше всего приходит пользователей из США и они лучше других конвертируется. При этом большая часть пользователей заходит с мобильных устройств, это тоже стоит отметить. Также, мы можем сказать, что наибольшую конверсию имеют пользователи, которые пользуются Mac, а затем следуют пользователи iPhone, т.е. мы можем сказать, что в целом пользователи Apple имеют лучшую конверсию в покупателей. Возможно, тут есть плюсы ApplePay.

# Шаг 4. Маркетинг

Взглянем на совокупные траты на маркетинг.


In [ ]:
print('Траты на маркетинг по всем источникам:', round((costs['costs'].sum()),2))

Посмотрим, как это распределилось по источникам.

In [ ]:
costs_per_channel = costs.groupby('channel')['costs'].sum()
costs_per_channel = costs_per_channel.sort_values(ascending = False).reset_index()
costs_per_channel['cost_%'] = costs_per_channel['costs']/costs['costs'].sum()
costs_per_channel.style.format({('cost_%'):'{:.2%}'})

Больше всего бюджета аккумулируют TipTop и FaceBoom.

Посмторим траты на каждый источник в разбивке по месяцам.

In [ ]:
costs['month'] = pd.to_datetime(costs['dt']).dt.month
costs_per_month = costs.pivot_table(index='month', columns='channel', values='costs', aggfunc='sum')
costs_per_month

In [ ]:
costs['week'] = pd.to_datetime(costs['dt']).dt.isocalendar().week
costs_per_week = costs.pivot_table(index='week', columns='channel', values='costs', aggfunc='sum')
costs_per_week

In [ ]:
costs_per_month.plot(linewidth=3,
                     alpha=0.7,
                     figsize=(15,10),
                     grid = True) 
plt.xlabel('Месяц')
plt.ylabel('Сумма')

costs_per_week.plot(linewidth=3,
                    alpha=0.7,
                    figsize=(15,10),
                    grid = True) 
plt.xlabel('Неделя')
plt.ylabel('Сумма')

При сопоставлении с данными о конверсии, данные из таблицы о увеличение бюджета для TipTop и FaceBoom не дают должного результата, и напротив lambdaMediaAds при кратно меньших бюджетах дают результаты гораздо лучше.

Смотрим САС по каналам и динамику ее роста.

In [ ]:
report_cac = profiles.pivot_table(
    index='channel', values='acquisition_cost', aggfunc='mean'
).sort_values(by='acquisition_cost', ascending=False)

display(report_cac)

In [ ]:
profiles.pivot_table(index='dt',
                     columns='channel',
                     values='acquisition_cost',
                     aggfunc='mean').plot(grid=True, figsize=(14,7))
plt.ylabel('CAC, $')
plt.xlabel('Дата привлечения')
plt.title('Динамика САС по каналам привлечения')
plt.show()

Наиболее высокий САС у TipTop и он постоянно возрастал по мере развития событий. По остальным источникам САС практически не менялся.

# Оценить окупаемость рекламы для привлечения пользователей

С помощью LTV и ROI:
- Проанализируем `общую окупаемость рекламы`;
- Проанализируем окупаемость `рекламы с разбивкой по устройствам`;
- Проанализируем окупаемость `рекламы с разбивкой по странам`;
- Проанализируем окупаемость `рекламы с разбивкой по рекламным каналам`.

Опишем проблемы, которые обнаружили. Ответим на вопросы:
- Окупается ли реклама, направленная на привлечение пользователей в целом? 
- Какие устройства, страны и рекламные каналы могут оказывать негативное влияние на окупаемость рекламы?
- Чем могут быть вызваны проблемы окупаемости? Изучите конверсию и удержание с разбивкой по устройствам, странам, рекламным каналам.

Опишем возможные причины обнаруженных проблем и сформируем рекомендации для рекламного отдела. При решении этого шага считаем, что мы смотрите данные `1-го ноября 2019 года` и что в нашей организации принято считать, что окупаемость должна наступать не позднее, чем через 2 недели после привлечения пользователей.

In [ ]:
observation_date = datetime(2019, 10, 27).date()  
horizon_days = 14

Смотрим конверсию по регионам.

In [ ]:
conversion_raw, conversion_grouped, conversion_history = get_conversion(
    profiles, orders, observation_date, horizon_days, dimensions=['region']
)

plot_conversion(conversion_grouped, conversion_history, horizon_days) 

Видим, что конверсия пользователей из США кратно выше чем из осталных регионов, как по лайфтайму так и в динамике.

Смотрим удержание по регионам.

In [ ]:
retention_raw, retention_grouped, retention_history = get_retention(
    profiles, visits, observation_date, horizon_days, dimensions=['region']
)

plot_retention(retention_grouped, retention_history, horizon_days) 

In [ ]:
retention_grouped

С удержанием пользователей картина противоположная. Платящие пользователи из США показывают худший результат, нежели пользователи из Британии, Франции и Германии, как по лайфтайму так и в динамике.

Смотрим конверсию по каналам.

In [ ]:
conversion_raw, conversion_grouped, conversion_history = get_conversion(
    profiles, orders, observation_date, horizon_days, dimensions=['channel']
)

plot_conversion(conversion_grouped, conversion_history, horizon_days) 

Результаты каждого канала по лайфтайму колеблятся в пределах 0.1, что в целом не дает понимания, какой из них наиболее успешен или перспективен. Наиболее высокие значения показывают 'FaceBoom','AdNonSense' и 'lambdaMediaAds'. В динамике же только 'lambdaMediaAds', показывает значения немногим выше средних. 

Смотрим удержание по каналам.

In [ ]:
retention_raw, retention_grouped, retention_history = get_retention(
    profiles, visits, observation_date, horizon_days, dimensions=['channel']
)

plot_retention(retention_grouped, retention_history, horizon_days) 

В картине по лайфтайму видим, что 'FaceBoom' показывает наихудщий результат, схожий с органическими пользователями. 'WahooNetBanner', 'RocketsSuperAds' и 'lambdaMediaAds'показывают наилучший результат удержания пользователей.

Смотрим конверсию по устройствам.

In [ ]:
conversion_raw, conversion_grouped, conversion_history = get_conversion(
    profiles, orders, observation_date, horizon_days, dimensions=['device']
)

plot_conversion(conversion_grouped, conversion_history, horizon_days) 

Наибольшие значения показывает Mac, Android и iPhone, как по лайфтайму так и в динамике.

Смотрим удержание по устройствам.

In [ ]:
retention_raw, retention_grouped, retention_history = get_retention(
    profiles, visits, observation_date, horizon_days, dimensions=['device']
)

plot_retention(retention_grouped, retention_history, horizon_days) 

По лайфтайму наилучшее удержание платящих пользователей показывает PC. В динамике так же довольно высокие значения у Mac.

### Общая окупаемость рекламы

In [ ]:
observation_date = datetime(2019, 11, 1).date()  
horizon_days = 14
profiles = profiles.query('channel != "organic"')

Взглянем на значения LTV и ROI по всей совокупности данных.

In [ ]:
ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles, orders, observation_date, horizon_days
)

plot_ltv_roi(
    ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=14
) 

Наблюдаем, что динамика ROI за лайфтайм падает. При относительно стабильной динамике LTV, динамика САС растёт с мая по конец октября. Эту закономерность мы наблюдаем в динамике ROI, что при сильном увеличении САС, в равной степени падает динамика ROI пользователей.

### Окупаемость рекламы с разбивкой по устройствам

Взглянем на значения LTV и ROI с разбивкой по типу устройств.

In [ ]:
ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles, orders, observation_date, horizon_days, dimensions=['device']
)

plot_ltv_roi(
    ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=14
) 

Наиболее перспективно выглядят данные PC. А именно: минимальный CAC среди других устройств и лучшие показатели по ROI, как по лайфтайму так и в динамике. Это значит, что у нас, по крайней мере, нет технических проблем, влияющих на монетизацию.

### Окупаемость рекламы с разбивкой по странам

Взглянем на значения LTV и ROI с разбивкой по регионам.

In [ ]:
ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles, orders, observation_date, horizon_days, dimensions=['region']
)

plot_ltv_roi(
    ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=14
) 

В разбивке по регионам ситуация противоречивая. Есть фаворит по LTV - USA, но у этого региона плохие значения CAC, что портит значения ROI, а это ключевой показатель среди трех пересичсленных. У остальных трех регионов противоположная ситуация: значения LTV куда скромне, зато превосходные значения CAC и ROI значительно выше 1.0. 

### Окупаемость рекламы с разбивкой по рекламным каналам

Взглянем на значения LTV и ROI с разбивкой по рекламным каналам.

In [ ]:
ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles, orders, observation_date, horizon_days, dimensions=['channel']
)

plot_ltv_roi(
    ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=30
) 

Есть `три фаворита` по LTV: `lambdaMediaAds`, `TipTop`, `RocketSuperAds`. Однако, `TipTop` постоянно `наращивает САС`, чем значительно проигрывает конкурентам. Более того, если смотреть на ROI, то `TipTop не выходит на окупаемость`. По `ROI наилучшие результат` представляют `YRabbit`, `RocketSuperAds`, `MediaTornado` и `lambdaMediaAds`.

# Выводы

Если мы взглянем на общую картину по ROI, то видим что маркетинг в течении лайфтайма не окупается, а в динамике если и имеет положительные показатели на старте, то уже ко второму месяцу становится убыточным. 

Большая часть бюджета на маркетинг расходуется на две компании `FaceBoom` и `TipTop`, при этом эти источники `не окупаются`. Исходя из данных приведенных выше, `наиболее преспективными` выглядят две компании: `lambdaMediaAds`, `RocketSuperAds`. Они дают в совокупности `наилучшие показатели по конверсии, удержанию, LTV, CAC и ROI`. Было бы верным шагом `распределить часть долей` в рекламном бюджете `на lambdaMediaAds и RocketSuperAds` и проследить как будут меняться данные в дальнейшем.

В целом если исходить из сути приложения, оно - развлекательное, что означает импульсивное использование, а не планово-итеративное, и также скорее всего работает по подписке. Для того чтобы принебречь малыми показателями конверсии, стоит иметь довольно высокие значения по LTV. Так как важнее, чтобы 2 из 10 пользователей, которые пришли, остались и платили, чем иметь много one-time buyers. 

Рекомендовано проверить работу приложения на Mac, Android и iPhone на наличие технических сбоев. Так как они показывают отличные данные по конверсии, но сильно падают в показателях удержания. 

Так же стоит проверить насколько эффективно локализовано приложение для US. Данные по US так же показывают высокую конверсию, но слабое удержание.